In [ ]:
import re
import subprocess
import pathlib
import numpy
import mpmath

from lips import Particles

In [ ]:
script_directory = "/home/gdl/Programs/PentagonFunctions-cpp/build/python_interface"

In [ ]:
# script_directory = str(pathlib.Path(__file__).parent.resolve()) + "/../build/python_interface"

In [ ]:
contants = {
        'tci[1,1]': -1j * mpmath.pi,
        'tci[1,2]': 1j * mpmath.pi,
        'tci[2,1]': mpmath.mpmathify('-1.01494160640965362502120255427452028594168930753029979201748910677659747625824j'),
        'tci[3,1]': mpmath.mpmathify('-0.57068163536563662179769055067238392601963107788880633412884021759219376534843j'),
        'tci[3,2]': mpmath.mpmathify('-1.95805792209958948875223725953776226311620663441329003133540714588177706642244j'),
        'tci[4,1]': mpmath.mpmathify('-0.9158468848305221005784557354615554066325218024536092822321305122909406032199j'),
        'tci[4,2]': mpmath.mpmathify('-0.57506902367961493370903615395302608738483177035427241988595161280872934244094j'),
        'tci[4,3]': mpmath.mpmathify('-7.63587387430075836809558431362434791200091165900221605545744819859116757903429j'),
        'tcr[1,1]': mpmath.mpmathify('1.09861228866810969139524523692252570464749055782274945173469433363749429321861'),
        'tcr[1,2]': mpmath.mpmathify('0.6931471805599453094172321214581765680755001343602552541206800094933936219697'),
        'tcr[2,1]': mpmath.mpmathify('0.83327188647738995744101246196890039744072476234026111025888050483590803187341'),
        'tcr[3,1]': mpmath.mpmathify('0.73806064483085791066377614636565032547367306985351329858054523307043380943959'),
        'tcr[3,2]': mpmath.mpmathify('0.25846139579657330528800012987367261202162535352798804747584081415085729951564'),
        'tcr[3,3]': mpmath.mpmathify('1.20205690315959428539973816151144999076498629234049888179227155534183820558957'),
        'tcr[4,1]': mpmath.mpmathify('0.34079113085625075247764094401220231521228085828689962786864627412151691807202'),
        'tcr[4,2]': mpmath.mpmathify('0.51747906167389938633075816189886294562237747514137925824431934797700828158186'),
        'tcr[4,3]': mpmath.mpmathify('0.2541161907463435340596737131535205829398478245574739638781062235368841219817'),
        'tcr[4,4]': mpmath.mpmathify('0.69919359449299809540702260849718121951534934751545290997293176683694496096964'),
        'tcr[4,5]': mpmath.mpmathify('0.79222102797282777952948578955735741116739480858778316087280447831968760156385')
}

In [ ]:
oddFs = [[[3, 1], [3, 2], [3, 3], [3, 4], [3, 5]], 
         [[2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9]], 
         [[23], [25], [26], [43], [45], [46], [62], [63], [64], [70], [71], [72], [78], [80], [81], [82], [83], [88], [93], [98], [103]], 
         [[25], [28], [39], [41], [45], [49], [51], [54], [61], [63], [64], [80], [83], [91], [93], [96], [100], [102], [105], [111], [113], [114], [129], [132], [139], [141], [142], [146], [148], [151], [156], [158], [159], [171], [174], [182], [184], [188], [190], [192], [195], [199], [201], [202], [213], [215], [219], [221], [225], [229], [233], [234], [235], [244], [246], [250], [252], [255], [259], [262], [263], [271], [273], [277], [279], [282], [286], [289], [290], [291], [293], [295], [299], [302], [303], [309], [313], [315], [316], [317], [325], [330], [331], [332], [333], [334], [335], [336], [337], [338], [339], [340], [343], [346], [348], [349], [350], [353], [356], [358], [359], [360], [363], [366], [368], [369], [371], [372], [373], [375], [376], [377], [378], [379], [380], [381], [382], [383], [384], [385], [386], [387], [388], [389], [390], [393], [396], [398], [399], [400], [403], [406], [408], [409], [411], [412], [413], [414], [415], [416], [417], [418], [419], [420], [421], [422], [425], [428], [429], [430], [431], [432], [433], [434], [435], [436]]]
oddtcis = [[1],[1],[1,2],[1,2,3]]

In [ ]:
def evaluate_pentagon_functions(pentagon_monomials, phase_space_point, 
                                pentagon_function_set=["m0", "m1"][0], precision=["d", "q", "o"][0], 
                                number_of_cores=8, verbose=False):
    assert precision in ["d", "q", "o"]
    assert pentagon_function_set in ["m0", "m1"]
    assert isinstance(number_of_cores, int)
    # set precision in mpmath
    if precision == "d":
        mpmath.mp.dps = 16
    elif precision == "q":
        mpmath.mp.dps = 32
    elif precision == "o":
        mpmath.mp.dps = 64
    # build pentagon string of indices, eg: 1 1 1;1 1 2;E
    pentagon_monomials_as_indices = ["{" + ",".join(re.findall("(\d+)", entry)) + "}" for entry in pentagon_monomials if "F" in entry]
    pentagon_input_string = ";".join(pentagon_monomials_as_indices).replace("{", "").replace("}", "").replace(",", " ") + ";E"
    # build mandelstams - drop imaginary part, it should be numerically small, if at all present
    s12, s23, s34, s45, s15 = phase_space_point("s12"), phase_space_point("s23"), phase_space_point("s34"), phase_space_point("s45"), phase_space_point("s15")
    s12, s23, s34, s45, s15 = [mpmath.mpf(mandel.real) for mandel in [s12, s23, s34, s45, s15]]
    # call PentagonFunctions++ via pyInterface.cpp
    PentagonFunctions_cppInterface = subprocess.Popen(
        [f"./pyInterface"] + [pentagon_function_set, precision, str(number_of_cores)], 
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, 
        cwd=script_directory)
    PentagonFunctions_cppInterface.stdin.write(pentagon_input_string.encode())
    PentagonFunctions_cppInterface.stdin.write(f"{s12} {s23} {s34} {s45} {s15}".encode())
    stdout, stderr = PentagonFunctions_cppInterface.communicate()
    stdout, stderr = stdout.decode(), stderr.decode()
    if verbose:
        print(stdout)
        print(stderr)
    # do some light parsing
    content = re.sub(r"`\d+(\.\d*){0,1}", "", stdout)
    content = content.replace("*I", "j").replace("I*", "1j*").replace("*^", "e").replace("^", "**").replace("{", "").replace("}", "")
    content = re.sub(r"1j\*([\-\+\d\.e]*)", r"\1j", content).replace(" ", "").replace("+-", "-").replace("\n", "")
    numerical_result = [mpmath.mpmathify(entry) for entry in content.split(",")]
    numerical_pentagon_dict = dict(zip(pentagon_monomials, numerical_result))
    numerical_pentagon_dict = {**numerical_pentagon_dict, **contants}
    return numerical_pentagon_dict

In [ ]:
def fix_parity_odd(numerical_pentagon_dict, phase_space_point):
    if (1j * oPsBenchmark("tr5_1234")).real < 0:
        # flip odd F's
        for weight, l_odd_pentagon_indices in enumerate(oddFs):
            for odd_pentagon_indices in l_odd_pentagon_indices:
                function_name = f'F[{weight+1},' + ','.join(map(str, odd_pentagon_indices)) + ']'
                if function_name in numerical_pentagon_dict.keys():
                    numerical_pentagon_dict[function_name] = -numerical_pentagon_dict[function_name]
    else:
        # flip odd tci's
        for weight, l_odd_pentagon_indices in enumerate(oddtcis):
            for odd_pentagon_index in l_odd_pentagon_indices:
                function_name = f'tci[{weight+1},{odd_pentagon_index}]'
                if function_name in numerical_pentagon_dict.keys():
                    numerical_pentagon_dict[function_name] = -numerical_pentagon_dict[function_name]
    return numerical_pentagon_dict

In [ ]:
oPsBenchmark = Particles(5, real_momenta=True)
oPsBenchmark[1].four_mom = numpy.array([(-0.575+0j), (-0.575+0j), 0j, 0j])
oPsBenchmark[2].four_mom = numpy.array([(-0.575+0j), (0.575+0j), 0j, 0j])
oPsBenchmark[3].four_mom = numpy.array([(0.4588582395652173+0j), (0.405584802173913+0j), (0.20777834301052356+0j), (-0.05366574734632376+0j)])
oPsBenchmark[4].four_mom = numpy.array([(0.23112940869565216+0j), (-0.09707956260869566+0j), (0.009377939347234585+0j), (-0.20954335193774518+0j)])
oPsBenchmark[5].four_mom = numpy.array([(0.46001235173913047+0j), (-0.3085052395652174+0j), (-0.2171562823577582+0j), (0.263209099284069+0j)])

In [ ]:
numerical_pentagon_dict = evaluate_pentagon_functions(['F[3,25]',
 'F[4,347]',
 'F[3,22]',
 'F[4,115]',
 'F[3,1]',
 'F[4,122]',
 'F[4,365]'], oPsBenchmark, precision="d", verbose=False)

In [ ]:
numerical_pentagon_dict = fix_parity_odd(numerical_pentagon_dict, oPsBenchmark)

In [ ]:
numerical_pentagon_dict

### trying pExpect

In [ ]:
import pexpect

In [ ]:
pentagon_monomials = ['F[3,25]',
 'F[4,347]',
 'F[3,22]',
 'F[4,115]',
 'F[3,1]',
 'F[4,122]',
 'F[4,365]']
phase_space_point = oPsBenchmark
pentagon_function_set=["m0", "m1"][0]
precision=["d", "q", "o"][0]
number_of_cores=8
verbose=False

In [ ]:
script_directory = "/home/gdl/Programs/PentagonFunctions-cpp/build"

In [ ]:
assert precision in ["d", "q", "o"]
assert pentagon_function_set in ["m0", "m1"]
assert isinstance(number_of_cores, int)
# set precision in mpmath
if precision == "d":
    mpmath.mp.dps = 16
elif precision == "q":
    mpmath.mp.dps = 32
elif precision == "o":
    mpmath.mp.dps = 64
# build pentagon string of indices, eg: 1 1 1;1 1 2;E
pentagon_monomials_as_indices = ["{" + ",".join(re.findall("(\d+)", entry)) + "}" for entry in pentagon_monomials if "F" in entry]
pentagon_input_string = ";".join(pentagon_monomials_as_indices).replace("{", "").replace("}", "").replace(",", " ") + ";E"
# build mandelstams - drop imaginary part, it should be numerically small, if at all present
s12, s23, s34, s45, s15 = phase_space_point("s12"), phase_space_point("s23"), phase_space_point("s34"), phase_space_point("s45"), phase_space_point("s15")
s12, s23, s34, s45, s15 = [mpmath.mpf(mandel.real) for mandel in [s12, s23, s34, s45, s15]]
# call PentagonFunctions++ via pyInterface.cpp
os.chdir(script_directory)
PentagonFunctions_cppInterface = pexpect.spawn(
    f"./pentagon_functions_evaluator_math " + " ".join([pentagon_function_set, precision, str(number_of_cores)]), 
    cwd=script_directory)

In [ ]:
pentagon_input_string

In [ ]:
PentagonFunctions_cppInterface.sendline(pentagon_input_string)

In [ ]:
PentagonFunctions_cppInterface.expect("Constructed")

In [ ]:
PentagonFunctions_cppInterface.sendline(f"{s12} {s23} {s34} {s45} {s15}")

In [ ]:
PentagonFunctions_cppInterface.expect("{")

In [ ]:
print(PentagonFunctions_cppInterface.before.decode())

In [ ]:

PentagonFunctions_cppInterface.stdin.write(pentagon_input_string)
PentagonFunctions_cppInterface.stdin.write(f"{s12} {s23} {s34} {s45} {s15}")
stdout, stderr = PentagonFunctions_cppInterface.communicate()
stdout, stderr = stdout.decode(), stderr.decode()

In [ ]:
s.expect("root@")
s.sendline("cd ~")

In [ ]:
def evaluate_pentagon_functions(pentagon_monomials, phase_space_point, 
                                pentagon_function_set=["m0", "m1"][0], precision=["d", "q", "o"][0], 
                                number_of_cores=8, verbose=False):
    assert precision in ["d", "q", "o"]
    assert pentagon_function_set in ["m0", "m1"]
    assert isinstance(number_of_cores, int)
    # set precision in mpmath
    if precision == "d":
        mpmath.mp.dps = 16
    elif precision == "q":
        mpmath.mp.dps = 32
    elif precision == "o":
        mpmath.mp.dps = 64
    # build pentagon string of indices, eg: 1 1 1;1 1 2;E
    pentagon_monomials_as_indices = ["{" + ",".join(re.findall("(\d+)", entry)) + "}" for entry in pentagon_monomials if "F" in entry]
    pentagon_input_string = ";".join(pentagon_monomials_as_indices).replace("{", "").replace("}", "").replace(",", " ") + ";E"
    # build mandelstams - drop imaginary part, it should be numerically small, if at all present
    s12, s23, s34, s45, s15 = phase_space_point("s12"), phase_space_point("s23"), phase_space_point("s34"), phase_space_point("s45"), phase_space_point("s15")
    s12, s23, s34, s45, s15 = [mpmath.mpf(mandel.real) for mandel in [s12, s23, s34, s45, s15]]
    # call PentagonFunctions++ via pyInterface.cpp
    PentagonFunctions_cppInterface = pexpect.spawn(
        [f"./pyInterface"] + [pentagon_function_set, precision, str(number_of_cores)], 
        stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, 
        cwd=script_directory)
    PentagonFunctions_cppInterface.stdin.write(pentagon_input_string.encode())
    PentagonFunctions_cppInterface.stdin.write(f"{s12} {s23} {s34} {s45} {s15}".encode())
    stdout, stderr = PentagonFunctions_cppInterface.communicate()
    stdout, stderr = stdout.decode(), stderr.decode()
    if verbose:
        print(stdout)
        print(stderr)
    # do some light parsing
    content = re.sub(r"`\d+(\.\d*){0,1}", "", stdout)
    content = content.replace("*I", "j").replace("I*", "1j*").replace("*^", "e").replace("^", "**").replace("{", "").replace("}", "")
    content = re.sub(r"1j\*([\-\+\d\.e]*)", r"\1j", content).replace(" ", "").replace("+-", "-").replace("\n", "")
    numerical_result = [mpmath.mpmathify(entry) for entry in content.split(",")]
    numerical_pentagon_dict = dict(zip(pentagon_monomials, numerical_result))
    numerical_pentagon_dict = {**numerical_pentagon_dict, **contants}
    return numerical_pentagon_dict

In [ ]:
numerical_pentagon_dict = evaluate_pentagon_functions(['F[3,25]',
 'F[4,347]',
 'F[3,22]',
 'F[4,115]',
 'F[3,1]',
 'F[4,122]',
 'F[4,365]'], oPsBenchmark, precision="d", verbose=False)